In [274]:
import numpy as np#

from pandas import DataFrame#
from pandas import read_csv#

from sklearn.model_selection import train_test_split#
from sklearn.linear_model import LinearRegression#
from sklearn.linear_model import LogisticRegression#

from sklearn.tree import DecisionTreeClassifier#
from sklearn.tree import DecisionTreeRegressor#

from sklearn.neighbors import KNeighborsClassifier#
from sklearn.neighbors import KNeighborsRegressor#


from sklearn.preprocessing import normalize as Normalize#
from sklearn.preprocessing import StandardScaler#
from sklearn.preprocessing import MinMaxScaler#

from sklearn.preprocessing import OneHotEncoder#

from sklearn.compose import make_column_transformer #

from warnings import filterwarnings as message#
from random import random as random#

message('ignore')

In [211]:
df=DataFrame(read_csv('attention.csv')).drop('Unnamed: 0', axis=1)
df=df.dropna(axis=0)
df.index=range(len(df.index))
df['company_working']=df[df.columns[0]]
df['score']=[int(element) for element in df['score']]
df=df.drop(df.columns[0], axis=1)
df.head()

,attention,solutions,score,company_working
0,divided,1,2,1
1,divided,1,3,2
2,divided,1,3,3
3,divided,1,5,4
4,divided,1,4,5


In [213]:
[df[column].unique() for column in df.columns]

[array(['divided', 'focused'], dtype=object),
 array([1, 2, 3], dtype=int64),
 array([2, 3, 5, 4, 6, 8, 7, 9], dtype=int64),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20], dtype=int64)]

In [214]:
df.head()

,attention,solutions,score,company_working
0,divided,1,2,1
1,divided,1,3,2
2,divided,1,3,3
3,divided,1,5,4
4,divided,1,4,5


In [234]:
DataFrame(Normalize(df)).head()

,0,1,2,3,4
0,0.377964,0.0,0.755929,0.377964,0.377964
1,0.258199,0.0,0.774597,0.516398,0.258199
2,0.223607,0.0,0.670820,0.670820,0.223607
3,0.152499,0.0,0.762493,0.609994,0.152499
4,0.152499,0.0,0.609994,0.762493,0.152499


In [240]:
DataFrame(StandardScaler().fit_transform(df)).head()

,0,1,2,3,4
0,1.0,-1.0,-2.443039,-1.647509,-1.224745
1,1.0,-1.0,-1.824548,-1.474087,-1.224745
2,1.0,-1.0,-1.824548,-1.300665,-1.224745
3,1.0,-1.0,-0.587566,-1.127243,-1.224745
4,1.0,-1.0,-1.206057,-0.953821,-1.224745


In [241]:
DataFrame(MinMaxScaler().fit_transform(df)).head()

,0,1,2,3,4
0,1.0,0.0,0.000000,0.000000,0.0
1,1.0,0.0,0.142857,0.052632,0.0
2,1.0,0.0,0.142857,0.105263,0.0
3,1.0,0.0,0.428571,0.157895,0.0
4,1.0,0.0,0.285714,0.210526,0.0


In [259]:
x,y=df.drop('solutions', axis=1), df['solutions']
x=DataFrame(StandardScaler().fit_transform(x))
x_train,x_test, y_train,y_test=train_test_split(x , y, test_size=0.2 , random_state=1961, shuffle=True)

In [260]:
model=LogisticRegression()
model.fit(x_train, y_train)
model.coef_

array([[-0.15169063,  0.15169063, -0.59396793, -0.12334706],
       [-0.04314718,  0.04314718,  0.04618692,  0.16719352],
       [ 0.19483781, -0.19483781,  0.547781  , -0.04384646]])

In [272]:
[model.score(x_train,y_train),model.score(x_test,y_test)]

[0.4375, 0.08333333333333333]

In [215]:
df['attention'].unique()

array(['divided', 'focused'], dtype=object)

In [216]:
class data_reader:
    def preprocessor_columns(_df_, _column_):
        column_cut=make_column_transformer((OneHotEncoder(), [_column_]))
        _df_=column_cut.fit_transform(df)
        return DataFrame(_df_, columns=[column_name.split('_')[-1] for column_name in column_cut.get_feature_names()])

    def preprocessor_data(_df_):
        return preprocessor_columns(_df_=df, _column_='attention').join(df.T[2:].T).join(DataFrame(df['solutions']))

In [217]:
df=data_reader.preprocessor_data(_df_=df)

In [218]:
df.head()

,divided,focused,score,company_working,solutions
0,1.0,0.0,2,1,1
1,1.0,0.0,3,2,1
2,1.0,0.0,3,3,1
3,1.0,0.0,5,4,1
4,1.0,0.0,4,5,1


In [219]:
def Model_unsamble(_df_, _column_, _model_type_='model_type'):
    x,y=df.drop(_column_, axis=1), df[_column_]
    x_train,x_test, y_train,y_test=train_test_split(x , y, test_size=0.2 , random_state=1961, shuffle=True)
    
    if _model_type_=='LinearRegression':
        model=LinearRegression()
        
    elif _model_type_=='DecisionTreeClassifier':
        model=DecisionTreeClassifier()
    
    elif _model_type_=='DecisionTreeRegressor':
        model=DecisionTreeRegressor()
    
    elif _model_type_=='KNeighborsClassifier':
        model=KNeighborsClassifier()
        
    elif _model_type_=='KNeighborsRegressor':
        model=KNeighborsRegressor()
        
    else:
        return 'errors'
    
    model.fit(x_train, y_train)
    return [model.score(x_train,y_train),model.score(x_test,y_test)]

In [220]:
DataFrame(np.array(
    [Model_unsamble(_df_=df, _column_=column, _model_type_='LinearRegression') for column in df.columns])
          , columns=['train_score', 'test_score'])

,train_score,test_score
0,1.000000,1.000000
1,1.000000,1.000000
2,0.373201,0.259973
3,0.743458,0.760999
4,0.177950,-0.073544


In [221]:
DataFrame(np.array(
    [Model_unsamble(_df_=df, _column_=column, _model_type_='DecisionTreeClassifier'
                   ) if len(df[column].unique())<5 else [None,None
                            ] for column in df.columns]), columns=['train_score', 'test_score'])

,train_score,test_score
0,1,1
1,1,1
2,None,None
3,None,None
4,0.895833,0.25


In [222]:
DataFrame(np.array(
    [Model_unsamble(_df_=df, _column_=column, _model_type_='DecisionTreeClassifier'
                   ) if len(df[column].unique())<5 else [None,None
                                ] for column in df.columns]), columns=['train_score', 'test_score'])

,train_score,test_score
0,1,1
1,1,1
2,None,None
3,None,None
4,0.895833,0.166667


In [223]:
DataFrame(np.array(
    [Model_unsamble(_df_=df, _column_=column, _model_type_='KNeighborsClassifier'
                ) if len(df[column].unique())<5 else [None,None
                            ] for column in df.columns]), columns=['train_score', 'test_score'])

,train_score,test_score
0,1,0.916667
1,1,0.916667
2,None,None
3,None,None
4,0.541667,0.0833333


In [224]:
DataFrame(np.array(
    [Model_unsamble(_df_=df, _column_=column, _model_type_='KNeighborsRegressor'
                ) for column in df.columns]), columns=['train_score', 'test_score'])

,train_score,test_score
0,0.942937,0.865000
1,0.942937,0.865000
2,0.517385,0.534802
3,0.756600,0.679534
4,0.257566,-0.426542
